# Kayas Assistant — QLoRA Fine-tuning on Kaggle

Use this notebook to fine-tune an instruct model on your enhanced 8k JSONL dataset with QLoRA.

Prerequisites (in Kaggle Notebook settings):
- Accelerator: GPU (T4/P100)
- Internet: On (for pip and model download)
- Add Data: Attach your dataset (e.g., a Kaggle Dataset or uploaded file) containing `mega_brain_dataset_8000_enhanced.jsonl`.

Outputs will be saved under `/kaggle/working/brain-lora`.

In [ ]:
# Environment check
import os, sys, platform, shutil
print('Python:', sys.version)
print('OS:', platform.platform())

# Optional: list input data directory
INPUT_DIR = '/kaggle/input'
if os.path.exists(INPUT_DIR):
    for root, dirs, files in os.walk(INPUT_DIR):
        print('Found in /kaggle/input:', root)
        # Only show top-level to keep output concise
        break
else:
    print('Warning: /kaggle/input not found (are you running locally or forgot to attach data?)')

In [ ]:
# Install dependencies
%pip install -q -U transformers datasets peft trl accelerate bitsandbytes einops safetensors huggingface_hub

## Configuration
- Default base model is a small, Kaggle-friendly instruct model.
- You can switch to a larger model if you have time/VRAM (e.g., Mistral-7B), but 7B can be slow or memory-constrained on Kaggle.
- The notebook tries to auto-discover your data file under `/kaggle/input`.

In [ ]:
# Choose Qwen 7B Instruct (multi-GPU friendly with QLoRA)
# Alternatives for lower VRAM: 'Qwen/Qwen2.5-3B-Instruct', 'microsoft/Phi-3-mini-4k-instruct'
from pathlib import Path
import random

BASE_MODEL = 'Qwen/Qwen2.5-7B-Instruct'

OUTPUT_DIR = Path('/kaggle/working/brain-lora')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# If you know the exact path, set it here; otherwise auto-discovery will run below.
DATASET_FILE_OVERRIDE = ''  # e.g., '/kaggle/input/your-dataset/mega_brain_dataset_8000_enhanced.jsonl'

SEED = 42
random.seed(SEED)

print('Base model:', BASE_MODEL)
print('Output dir:', str(OUTPUT_DIR))

## Locate dataset under /kaggle/input
The code will auto-find `mega_brain_dataset_8000_enhanced.jsonl` if you attached it as a Kaggle Dataset or uploaded it via Add Data.
You can also set `DATASET_FILE_OVERRIDE` to an explicit path.

In [ ]:
from pathlib import Path
import os

def find_dataset_file(filenames=None) -> str:
    # Allow explicit override via cell variable or env var
    if DATASET_FILE_OVERRIDE and Path(DATASET_FILE_OVERRIDE).exists():
        return DATASET_FILE_OVERRIDE
    env_override = os.environ.get('KAGGLE_DATASET_FILE', '')
    if env_override and Path(env_override).exists():
        return env_override

    base = Path('/kaggle/input')
    if not base.exists():
        return ''

    # Try common enhanced filenames first, then fall back
    candidates_to_try = filenames or [
        'mega_brain_dataset_10000_enhanced.jsonl',
        'mega_brain_dataset_8000_enhanced.jsonl',
        'mega_brain_dataset_10000.jsonl',
        'mega_brain_dataset_8000.jsonl',
    ]
    for name in candidates_to_try:
        found = list(base.rglob(name))
        if found:
            return str(found[0])

    # Fallback: any mega_brain_dataset*.jsonl under input
    any_found = list(base.rglob('mega_brain_dataset_*.jsonl'))
    return str(any_found[0]) if any_found else ''

DATA_PATH = find_dataset_file()
print('Discovered dataset path:' , DATA_PATH if DATA_PATH else 'NOT FOUND')
assert DATA_PATH, 'Dataset file not found. Attach data via "Add Data" and ensure a mega_brain_dataset_*.jsonl file exists, or set DATASET_FILE_OVERRIDE.'

## Load and prepare dataset
The dataset is JSONL with one object per line containing a `messages` array (system/user/assistant). The notebook will auto-find `mega_brain_dataset_8000_enhanced.jsonl` under `/kaggle/input` if attached via Add Data.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# T4 prefers fp16 compute for 4-bit quant
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

# Re-load tokenizer here as each process may re-initialize
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

# IMPORTANT: For DDP, do NOT set device_map here
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
)
model.config.use_cache = False  # safer with gradient checkpointing
print('Model loaded (4-bit, fp16 compute).')

## Load model and tokenizer (4-bit)
We use 4-bit quantization (bitsandbytes) plus LoRA to train within Kaggle limits.

## Configure LoRA and SFT Trainer
We format chat samples using the tokenizer's chat template and train with TRL's SFTTrainer.

In [ ]:
from peft import LoraConfig
from trl import SFTTrainer
from transformers import TrainingArguments

# Common target modules for Qwen/LLaMA/Mistral families
LORA_TARGET_MODULES = ['q_proj','k_proj','v_proj','o_proj','gate_proj','up_proj','down_proj']
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM',
    target_modules=LORA_TARGET_MODULES,
)

# Formatting function: render messages using chat template
def formatting_func(example_batch):
    texts = []
    for msgs in example_batch['messages']:
        text = tokenizer.apply_chat_template(
            msgs, tokenize=False, add_generation_prompt=False
        )
        texts.append(text)
    return texts

# We'll rebuild model/tokenizer and dataset inside the train loop for multi-GPU
from datasets import Dataset, DatasetDict
import json, random

def train_loop():
    import torch
    from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
    from trl import SFTTrainer
    from transformers import TrainingArguments

    # Re-seed per process
    random.seed(SEED)

    # Load tokenizer/model (per process)
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.float16,
    )
    tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
    if tok.pad_token_id is None:
        tok.pad_token = tok.eos_token
    mdl = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        quantization_config=bnb_config,
    )
    mdl.config.use_cache = False

    # Load dataset again inside process
    def load_jsonl_messages(path: str):
        rows = []
        with open(path, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                obj = json.loads(line)
                if 'messages' in obj:
                    rows.append({'messages': obj['messages']})
        return rows

    raw_rows = load_jsonl_messages(DATA_PATH)
    random.shuffle(raw_rows)
    val_size = max(1, int(0.025 * len(raw_rows)))
    val_rows = raw_rows[:val_size]
    train_rows = raw_rows[val_size:]

    ds_train = Dataset.from_list(train_rows)
    ds_val = Dataset.from_list(val_rows)

    # Training args optimized for 2x T4 with QLoRA 7B
    training_args = TrainingArguments(
        output_dir=str(OUTPUT_DIR),
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=8,
        learning_rate=2e-4,
        logging_steps=10,
        num_train_epochs=2,
        warmup_ratio=0.03,
        fp16=True,
        bf16=False,
        lr_scheduler_type='cosine',
        gradient_checkpointing=True,
        save_strategy='no',
        evaluation_strategy='steps',
        eval_steps=250,
        optim='paged_adamw_8bit',
        group_by_length=True,
        ddp_find_unused_parameters=False,
        report_to=[],
    )

    trainer = SFTTrainer(
        model=mdl,
        tokenizer=tok,
        train_dataset=ds_train,
        eval_dataset=ds_val,
        peft_config=lora_config,
        formatting_func=lambda batch: [
            tok.apply_chat_template(m, tokenize=False, add_generation_prompt=False) for m in batch['messages']
        ],
        max_seq_length=1024,
        packing=True,
        args=training_args,
    )

    train_result = trainer.train()

    # Save only on main process
    if trainer.accelerator.is_main_process:
        trainer.model.save_pretrained(str(OUTPUT_DIR))
        tok.save_pretrained(str(OUTPUT_DIR))
    return train_result

## Train
This may take a few hours depending on the GPU and dataset size.

In [ ]:
# Kick off training (uses Accelerate under the hood; will leverage multiple GPUs if available)
result = train_loop()
result

## Save adapter and tokenizer
The LoRA adapter and tokenizer are saved under `/kaggle/working/brain-lora`.

In [ ]:
# Models and tokenizer are already saved inside train_loop on main process
from pathlib import Path
print('Artifacts present:', list(Path(OUTPUT_DIR).glob('*')))
print('Saved to', str(OUTPUT_DIR))

## Quick inference test
Load the base model with the trained adapter and generate a short response.

In [ ]:
from peft import PeftModel
from transformers import TextIteratorStreamer
import threading, time

# Reload for inference (4-bit + adapter)
base = AutoModelForCausalLM.from_pretrained(BASE_MODEL, quantization_config=bnb_config, device_map='auto')
tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tok.pad_token_id is None:
    tok.pad_token = tok.eos_token
inf_model = PeftModel.from_pretrained(base, str(OUTPUT_DIR))
inf_model.eval()

# Sample conversation
messages = [
    { 'role': 'system', 'content': 'You are a helpful assistant that can plan actions and clarify ambiguity.' },
    { 'role': 'user', 'content': 'open vscode then search repo readme for setup' }
]
prompt = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tok([prompt], return_tensors='pt').to(inf_model.device)

with torch.inference_mode():
    gen = inf_model.generate(**inputs, max_new_tokens=128, do_sample=True, temperature=0.7, top_p=0.9)
print(tok.decode(gen[0], skip_special_tokens=True))

## Notes
- For faster runs, reduce `max_seq_length` to 768 and/or `num_train_epochs` to 1.
- If you have more time/VRAM, try `num_train_epochs=2` or switch `BASE_MODEL` to a larger instruct model.
- To export, download `/kaggle/working/brain-lora` as an artifact or push to the Hugging Face Hub if you add a token.